# Lesson 4: Persistence and Streaming

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:") # in memory database! - can be leverages to use postgres and redis

In [10]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        # Checkpointer adds persistance! - Using SQL lite - in memory DB.. 
            # Can be connected to external DB - redis/postgres.. 
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [11]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [15]:
messages = [HumanMessage(content="What is the weather in Melbourne victoria?")]

In [16]:
thread = {"configurable": {"thread_id": "1"}}

In [17]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2gD6Aa6ltvg8T3rpzPjb1fES', 'function': {'arguments': '{"query":"current weather in Melbourne Victoria"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 748, 'total_tokens': 770}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_d576307f90', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4f45a883-00a0-47b8-bf26-9dd46f657e6c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Melbourne Victoria'}, 'id': 'call_2gD6Aa6ltvg8T3rpzPjb1fES'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Melbourne Victoria'}, 'id': 'call_2gD6Aa6ltvg8T3rpzPjb1fES'}
Back to the model!
[ToolMessage(content="HTTPError('502 Server Error: Bad Gateway for url: https://api.tavily.com/search')", name='tavily_search_results_json', tool_call_id='call_2gD6Aa6ltvg8T3rpz

In [20]:
messages = [HumanMessage(content="What about in perth?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_31GYvrI0OOtwyGM3iQl2J6Qo', 'function': {'arguments': '{"query":"current weather in Perth, WA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 2159, 'total_tokens': 2182}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_ce0793330f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9bf264aa-3c79-4e35-b3c2-1c59f4635a4d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Perth, WA'}, 'id': 'call_31GYvrI0OOtwyGM3iQl2J6Qo'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Perth, WA'}, 'id': 'call_31GYvrI0OOtwyGM3iQl2J6Qo'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Perth\', \'region\': \'Western Australia\', \'country\': \'Australi

In [21]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Between Melbourne, Sydney, and Perth, the current temperatures are as follows:\n\n- **Melbourne:** 12.4°C (54.3°F)\n- **Sydney:** Please check [AccuWeather](https://www.accuweather.com/en/au/sydney/22889/weather-forecast/22889) for the latest temperature.\n- **Perth:** 14.0°C (57.2°F)\n\nBased on the provided data, **Perth** is currently the warmest with a temperature of 14.0°C (57.2°F).', response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 2762, 'total_tokens': 2878}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_4008e3b719', 'finish_reason': 'stop', 'logprobs': None}, id='run-743f0be2-2b3f-48c8-87ad-7c4068159a3f-0')]}


In [22]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please clarify what you are comparing for warmth? Are you referring to the current temperatures of specific locations, the warmth of different materials, or something else?', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 149, 'total_tokens': 182}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_ce0793330f', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc0eb814-7e20-4f89-a6e2-7880fc29f3ab-0')]}


## Streaming tokens

In [26]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [27]:
messages = [HumanMessage(content="What is the weather in Melbourne Aus?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Melbourne Australia'}, 'id': 'call_gfHMGiDA5ntnWNOesSh2OBt0'}
Back to the model!
